In [1]:
# libraries
import os
import glob
import zipfile
import pandas as pd
import numpy as np
import re

In [2]:
# load and unzip files
path = os.getenv("HOME") + '/saturi_lab_multi_nmt_low_resource/data/raw/validation'
zipfiles = ['[라벨]강원도_학습데이터_2.zip', '[라벨]경상도_학습데이터_2.zip', '[라벨]전라도_학습데이터_2.zip', '[라벨]제주도_학습용데이터_3.zip', '[라벨]충청도_학습데이터_2.zip']
regs = ['kw','gs','jd','jj','cc']

for zp,reg in zip(zipfiles, regs) :
    extract_file = path +'/' + zp
    val_dir = path + '/' + reg

    os.makedirs(val_dir,exist_ok=True)    

    if not len(os.listdir(val_dir)) :
        with zipfile.ZipFile(zp,'r') as zip_ref :
            zip_ref.extractall(val_dir)

In [3]:
# extract saturi exerpts only
def doc_preprocess(txt,reg):
    raw = []
    for doc in txt : 
        with open(doc,'r') as f :
            lines = f.readlines() # 한줄 한줄씩 파일 가져오기
            for line in lines :
                line = re.sub(r'\d\W','',line)
                match = re.search(r'\(\S+\)\/\(\S+\)',line) # 찾기 조건
                if match :
                    line = line.strip() # \n 부호 지우기
                    pair = ' '.join(re.findall(r'\(\S+\)\/\(\S+\)',line))
                    normal = re.sub(r'\(\S+\)\/','', line) #표준어
                    dialect = re.sub(r'\/\(\S+\)','', line) #사투리
                    raw.append((normal, dialect,reg,pair)) #( 표준어, 사투리, 지역tag, original 표준어/사투리 )
                    
    return raw

data = {}
for reg in regs :
    val_dir = path + '/' + reg
    txt_files = glob.glob(val_dir + '/*.txt')
    data[reg] = doc_preprocess(txt_files,reg)

In [8]:
# put into dataframe

a = pd.DataFrame(columns = ['text','dial','reg','pair'])

for reg in regs :
    b = pd.DataFrame(data[reg], columns = ['text','dial','reg','pair'])
    a = a.merge(b,how='outer')

a.to_csv(path + '/dataset_validation.csv',index=False) # save validation dataset

a.info()
a['reg'].value_counts() #check ouptput

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316789 entries, 0 to 316788
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    316789 non-null  object
 1   dial    316789 non-null  object
 2   reg     316789 non-null  object
 3   pair    316789 non-null  object
dtypes: object(4)
memory usage: 9.7+ MB


reg
jj    136097
kw     60471
jd     56155
cc     33167
gs     30899
Name: count, dtype: int64